In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# Example TripAdvisor hotel review page
base_url = "https://www.tripadvisor.com/Hotel_Review-g60763-d93542-Reviews-or{}-Hotel_Name-New_York_City_New_York.html"

data = []

for offset in range(0, 50, 5):  # Adjust the range for more pages (each page = 5 reviews approx)
    url = base_url.format(offset)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    reviews = soup.find_all('div', class_='review-container')

    for review in reviews:
        title = review.find('span', class_='noQuotes')
        text = review.find('p', class_='partial_entry')
        rating_tag = review.find('span', class_='ui_bubble_rating')
        date_tag = review.find('span', class_='ratingDate')

        # Extract rating value from class name (e.g., "bubble_50" = 5.0 stars)
        rating = None
        if rating_tag:
            class_str = rating_tag.get("class", [])
            for cls in class_str:
                if cls.startswith("bubble_"):
                    rating = int(cls.split("_")[1]) / 10.0

        data.append({
            "title": title.text.strip() if title else None,
            "text": text.text.strip() if text else None,
            "rating": rating,
            "publishedDate": date_tag.get("title") if date_tag and date_tag.has_attr("title") else None,
            "location": "New York City",  # Hardcoded from URL (can be parsed)
        })

    time.sleep(2)  # Be respectful of TripAdvisor's servers

# Convert to DataFrame and save
df = pd.DataFrame(data)
df.to_excel("tripadvisor_reviews_scraped.xlsx", index=False)
print("Scraping complete. Data saved to Excel.")


Scraping complete. Data saved to Excel.
